In [1]:
import cv2 as cv
#import common
import numpy as np

                                                #the following are to do with this interactive notebook code
%matplotlib inline 
from matplotlib import pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0) # this controls figure size in the notebook

In [7]:
def nothing(x):
    pass

# Fungsi trackbar untk proses masking
def masking(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    
    min_r = cv.getTrackbarPos("min_red", "TrackBars")
    min_g = cv.getTrackbarPos("min_green", "TrackBars")
    min_b = cv.getTrackbarPos("min_blue", "TrackBars")
    
    lower = np.array([min_r, min_g, min_b])
    upper = np.array([255,255,255])
    mask = cv.inRange(hsv, lower, upper)
    
    return mask, min_r, min_g, min_b

In [10]:

cap = cv.VideoCapture(0)
cv.namedWindow("TrackBars")

# Create TrackBar
## Minimum range
cv.createTrackbar("min_red", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_green", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_blue", "TrackBars", 0, 255, nothing)

## Maximum range

while True:
    _, frame = cap.read()
    mask, min_r, min_g, min_b = masking(frame)
    
    #cv.imshow("Sample", frame)
    cv.imshow("Sample", mask)
    if cv.waitKey(1) & 0xFF==ord("q"):
        break
        
cap.release()
cv.destroyAllWindows()

print("min red, min green, min blue: {0}, {1}, {2}".format(min_r, min_g, min_b))

min red, min green, min blue: 0, 125, 186


In [11]:
# Algorithm:
# Reading RGB image --> Converting to HSV --> Implementing Mask -->
# Finding Contour Points --> Extracting Rectangle Coordinates -->
# Drawing Bounding Box --> Putting Label


# Defining lower bounds and upper bounds of founded Mask
min_blue, min_green, min_red = 0, 125, 186  # Dapat dari cell atas!!

cap = cv.VideoCapture(0)
#cv.namedWindow("Binary frame with Mask")
while True:
    _, frame = cap.read()
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)    #(480, 640, 3) Convert to HSV
    mask = cv.inRange(frame_hsv, (min_blue, min_green, min_red), (255,255,255))    #(480, 640) Implementing MASK
    
    #Finding Countors
    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    
    #draw contours
    #drawing = np.zeros((np.array(mask).shape[0], np.array(mask).shape[1], 3), dtype=np.uint8)
    #for i in range(len(contours)):
        #colors = (np.random.randint(0,256), np.random.randint(0,256), np.random.randint(0,256))
        #cv.drawContours(drawing, contours, i, colors, 1, cv.LINE_8, hierarchy, 0)
    
    #Extracting rectangle coordinates
    if contours:
        #Rectangle
        x, y, w, h = cv.boundingRect(contours[0])
        cv.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        
        #Putting Label
        label = "Detected Object!!"
        cv.putText(frame, label, (x-5, y-5), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)        
    
    #cv.imshow('Contours', drawing)
    cv.imshow("Binary frame", frame)
    if cv.waitKey(1) & 0xFF==ord("q"):
        break

cap.release()
cv.destroyAllWindows()